In [ ]:
#@title ## <center>1. Download rich package & Mount Gdrive</center>

gdrive_mode = 'MOUNT' #@param ['MOUNT', 'UNMOUNT']
gdrive_type = 'SHARED DRIVE' #@param ['MY DRIVE', 'SHARED DRIVE']
gdrive_shared_drive = 'my_shared_drive' #@param {type:"string"}
gdrive_subpath = "work" #@param {type:"string"}

if gdrive_type == "MY DRIVE":
    gdrive_path = f'/content/drive/My Drive'
else:
    gdrive_path = f'/content/drive/Shared drives/{gdrive_shared_drive}/{gdrive_subpath}'

!pip install rich > /dev/null

from rich import print
from rich.console import Console
from pathlib import Path

console = Console()
print(f'Installed rich package')

from google.colab import drive

drive.mount._DEBUG = False
if gdrive_mode == 'MOUNT':
    drive.mount('/content/drive', force_remount=True)
    # print(f'Google Drive mounted on "{gdrive_path}"')
elif gdrive_mode == 'UNMOUNT':
    try:
        drive.flush_and_unmount()
    except ValueError:
        pass
    get_ipython().system_raw('rm -rf /root/.config/Google/DriveFS')
    # print(f'Google Drive unmounted')

In [ ]:
#@title ##  <center>2. Download stuff</center>
#@markdown ### Download/Update ESRGAN:
# #@markdown ```Run this once. Running this while you already have a folder named Colab-ESRGAN in your Google Drive will give you an error.```
#@markdown The first time you have to select `CLONE`, the other times select `PULL`.
git_mode = 'PULL' #@param ['CLONE', 'PULL']
esrgan_dirname = "Colab-ESRGAN" #@param {type:"string"}

%cd "{gdrive_path}"
esrgan_path = f'{gdrive_path}/{esrgan_dirname}'
if git_mode == 'CLONE':
    !git clone https://github.com/efonte/ESRGAN {esrgan_path}
    %cd "{esrgan_path}"
else:
    %cd "{esrgan_path}"
    !git pull
with console.status("Installing packages..."):
    !pip install -q -r requirements.txt > /dev/null
    !apt-get -qq update > /dev/null
    !apt-get -qq install imagemagick > /dev/null
    # !apt-get -qq install parallel > /dev/null


In [ ]:
#@title ##  <center>Get available models</center>
#@markdown #### Get the models that are in the `{esrgan_path}/models` directory
models_path= Path(f'{esrgan_path}/models')
for model_path in models_path.glob("**/*.pth"):
    print(model_path.relative_to(models_path).as_posix())


In [ ]:
#@title ## <center>Upscale - Image Mode</center>
#@markdown #### ESRGAN image mode.

#@markdown ---
#@markdown ### Models:
#@markdown ##### Examples:<br>• `1x_model1.pth>4x_model2.pth`<br>• `1x_model1.pth@50|1x_model2.pth@50>2x_model3.pth`<br>• `1x_model1.pth:50&1x_model2.pth:50>2x_model3.pth`
models = "1x_NoiseToner_Uniform_100k.pth>4x_DigitalFrames_2.1_Aggressive.pth" #@param {type:"string"}
#@markdown ---
#@markdown ### Parameters:
#@markdown ##### Input folder:
input_path = "input/test" #@param {type:"string"}
#@markdown ##### Output folder:
output_path = "output/test" #@param {type:"string"}
#@markdown ##### Reverse Order:
reverse = False #@param {type:"boolean"}
#@markdown ##### Skip existing output files:
skip_existing = False #@param {type:"boolean"}
#@markdown ##### Delete input files after upscaling:
delete_input = False #@param {type:"boolean"}
#@markdown ##### Use FloatingPoint16/Halftensor type:
floating_point_16 = False #@param {type:"boolean"}
#@markdown ##### Helps seamlessly upscale an image:<br>`None` = disable seamless.<br>`tile` = repeating along edges.<br>`mirror` = reflected along edges.<br>`replicate` = extended pixels along edges.<br>`alpha_pad` = extended alpha border.
seamless = 'None' #@param ['None', 'tile', 'mirror', 'replicate', 'alpha_pad']
#@markdown ##### Caches the maximum recursion depth used by the split/merge function. Useful only when upscaling images of the same size:
cache_max_split_depth = False #@param {type:"boolean"}
#@markdown ##### Whether to use a 1 bit alpha transparency channel, Useful for PSX upscaling:
binary_alpha = False #@param {type:"boolean"}
#@markdown ##### Whether to use a 2 bit alpha transparency channel, Useful for PSX upscaling 
ternary_alpha = False #@param {type:"boolean"}
#@markdown ##### Only used when binary_alpha is supplied. Defines the alpha threshold for binary transparency:
alpha_threshold = 0.5 #@param {type:"number"}
#@markdown ##### Only used when binary_alpha is supplied. Determines the offset boundary from the alpha threshold for half transparency:
alpha_boundary_offset = 0.2 #@param {type:"number"}
#@markdown ##### Type of alpha processing to use:<br>`no_alpha` = no alpha processing.<br>`bas` = BA's difference method.<br>`alpha_separately` = upscaling the alpha channel separately (like IEU).<br>`swapping` = swapping an existing channel with the alpha channel.
alpha_mode ='alpha_separately' #@param ['no_alpha', 'bas', 'alpha_separately', 'swapping']
#@markdown ##### Use ImageMagick to save the upscaled image (higher quality but slower):
imagemagick = False #@param {type:"boolean"}
#@markdown ##### Convert the image to jpg:
jpg = False #@param {type:"boolean"}
#@markdown ##### Resize percent (`100` = no resize, `200` = x2, `50` = x0.5):
resize = 100 #@param {type:"number"}
#@markdown ##### Compress the output to zip file:
zip = False #@param {type:"boolean"}
#@markdown ##### Verbose mode:
verbose = False #@param {type:"boolean"}
#@markdown ---

esrgan_args = f'"{models}" --input "{input_path}" --output "{output_path}"'
if reverse:
    esrgan_args += ' --reverse'
if skip_existing:
    esrgan_args += ' --skip-existing'
if delete_input:
    esrgan_args += ' --delete-input'
if floating_point_16:
    esrgan_args += ' --floating-point-16'
if seamless != 'None':
    esrgan_args += f' --seamless {seamless}'
if cache_max_split_depth:
    esrgan_args += ' --cache-max-split-depth'
if binary_alpha:
    esrgan_args += ' --binary-alpha'
if ternary_alpha:
    esrgan_args += ' --ternary-alpha'
esrgan_args += f' --alpha-threshold {alpha_threshold} --alpha-boundary-offset {alpha_boundary_offset} --alpha-mode {alpha_mode}'
if jpg:
    esrgan_args += ' --jpg'
esrgan_args += f' --resize {resize}'
if imagemagick:
    esrgan_args += ' --imagemagick'
if zip:
    esrgan_args += ' --zip'
if verbose:
    esrgan_args += ' -v' 
%cd {esrgan_path}
# %run -i upscale.py image $esrgan_args > /dev/null
%run -i run.py image $esrgan_args


In [ ]:
#@title ## <center>ImageMagick - Utils</center>

#@markdown ---
#@markdown ##### Input folder:
input_path = "output/test" #@param {type:"string"}
#@markdown ##### Output folder (*must not exist*):
output_path = "output/test_resized" #@param {type:"string"}
#@markdown ##### Convert to jpg:
jpg = False #@param {type:"boolean"}
#@markdown ##### Resize percent (`100` = no resize, `200` = x2, `50` = x0.5):
resize = 80 #@param {type:"number"}
#@markdown ##### Show progress:
monitor = True #@param {type:"boolean"}
#@markdown ---

# !rm -r "{output_path}"
!mkdir "{output_path}"

mogrify_args = f'-resize {resize}%'
if monitor:
    mogrify_args += ' -monitor' 
if jpg:
    mogrify_args += ' -format jpg -sampling-factor 4:2:0 -strip -interlace JPEG -colorspace sRGB'
else:
    mogrify_args += ' -format png'

!mogrify {mogrify_args} -path "{output_path}" "{input_path}/*.png"
# !find "{input_path}" -type f -name '*.png' -print0 | xargs -0 -n3 mogrify {mogrify_args} -path "{output_path}"


In [ ]:
#@title ## <center>Upscale - Video Mode</center>
#@markdown #### ESRGAN video mode.

#@markdown ---
#@markdown ### Work in progress
#@markdown ---

print('TODO')